## Top Cell - working scraper which also outputs mobilities

In [1]:
#######################################################################################################
#######################################################################################################
#           this Python3 Program was written by Ben Coombs for use probing excel files                #
#              at CPI NetPark - Printable Electronics. It is not for general use                      #
#          and should be considered to be in beta-test mode. BAC Copyright March 2016.                #
#######################################################################################################
###########           ##############           #############         #################        #########

import openpyxl
from openpyxl import load_workbook
from openpyxl.cell import get_column_letter, column_index_from_string
import time
import os
import pandas as pd
import csv
import re

#start clock to calculate timings
start_time = time.clock()
      
# initiate some variables/lists - substrate is index so can count number of files searched thru	  
substrate = 0                                               
outputfile = []
channel_data = []
substrate_data = []
allVTOpositions = []
coords = []

def SearchXLWithRE(regex,worksheet):                                  # function to do a search over a
    for i in range(1, Row_limit):
        for j in range(1, Column_limit):
            query = worksheet.cell(row = i, column = j).value
            if type(query) == str:
                test = regex.search(query)
                if test:                                    # --> checks if regex has found a match
                    return [i,j]                            # --> row,column
    return [100,100]                                        # --> returns coordinates on sheet known to be BLANK
                                    
def DataSeeker(ORIGIN,rowOffset,colOffset,formatting,worksheet):
    result_raw = worksheet.cell(row=ORIGIN[0]+rowOffset+r,
                             column=ORIGIN[1]+colOffset).value
    if not type(result_raw) == str:
        result = formatting.format(result_raw)
    else:
        result = result_raw
    return result

def OriginFinder(Regex_Term,worksheet):
    Regex_Pattern = re.compile(Regex_Term, re.I)
    Origin_Name = SearchXLWithRE(Regex_Pattern,worksheet)
    return Origin_Name


def filterPick(list,filter):
    return [(m.group(1)) for l in list for m in (filter(l),) if m]


date = time.strftime('DIRECTORY TEST DATA OVERVIEW %Y-%m-%d.csv')#-%H%M%S)

with open(date, "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["File", "Date of test", "CL", "CW", "Mobility", "SDev", "On/Off", "VTO", "VTH", "Yield"])

    for root, dirs, files in os.walk(".", topdown=True):
        
        for fname in files:
            
            if fname.endswith('.xlsm') or fname.endswith('.xlsx'):
                
                wb_value = load_workbook(os.path.join(root, fname), data_only=True)
                names = wb_value.get_sheet_names()              
        
                if 'validated Summary Data' in names:

                    substrate = substrate + 1

                    wb_formula = load_workbook(os.path.join(root, fname))
                    ws1 = wb_value.get_sheet_by_name('validated Summary Data')
                    ws2 = wb_value.get_sheet_by_name('Data')
                    ws3 = wb_value.get_sheet_by_name('transferdata')
                    ws4 = wb_formula.get_sheet_by_name('validated Summary Data')

                    Row_limit = ws1.max_row
                    Column_limit = ws1.max_column
                    Col_limit = get_column_letter(Column_limit)

                    Mob1=[]
                    Mob2=[]
                    Mob3=[]
                    Mob4=[]
                    
                    for row in ws2.iter_rows('J5:J40'):
                        for cell in row:
                            Mob1.append(cell.value)
                            MobSS1 = Mob1
                    for row in ws2.iter_rows('J49:J84'):
                        for cell in row:
                            Mob2.append(cell.value)
                            MobSS2 = Mob2
                    for row in ws2.iter_rows('J93:J128'):
                        for cell in row:
                            Mob3.append(cell.value)
                            MobSS3 = Mob3                   
                    for row in ws2.iter_rows('J137:J172'):
                        for cell in row:
                            Mob4.append(cell.value)
                            MobSS4 = Mob4
                            
                    IndMobs = [MobSS1, MobSS2, MobSS3, MobSS4]
                    
                    for r in range(1,5):
                        VTO = DataSeeker(OriginFinder('^VTO$',ws1),0,0,"{:.2f}",ws4)    
                        VTO_tuple = (VTO)                      
                        allVTOpositions.append(VTO_tuple)           
                        continue
                        
                    seeker = re.compile("transferdata!(.*)\),").search
                    coords = filterPick(allVTOpositions,seeker)

                    VTO1 = []
                    vto1_form = tuple(ws3.iter_rows(coords[0]))
                    for row in vto1_form:
                        for cell in row:
                            VTO1.append(cell.value)
                    VTO2 = []
                    vto2_form = tuple(ws3.iter_rows(coords[1]))
                    for row in vto2_form:
                        for cell in row:
                            VTO2.append(cell.value)
                    VTO3 = []
                    vto3_form = tuple(ws3.iter_rows(coords[2]))
                    for row in vto3_form:
                        for cell in row:
                            VTO3.append(cell.value)
                    VTO4 = []
                    vto4_form = tuple(ws3.iter_rows(coords[3]))
                    for row in vto4_form:
                        for cell in row:
                            VTO4.append(cell.value)

                    IndVTOs = [VTO1, VTO2, VTO3, VTO4]


                    # for loop to return all data for the 4 subsites
                    # with offsets calculated from ORIGINS defined during these calls (using Origin Finder func)                

                    for r in range(1,5):

                        # these first three don't actually loop as are single admin-y data bits

                        test_date = ws1.cell(row=OriginFinder('Date',ws1)[0]+1,column=OriginFinder('Date',ws1)[1]).value
                        Comments = ws1.cell(row=OriginFinder('comment',ws1)[0],column=OriginFinder('comment',ws1)[1]+1).value
                        Capacitance = ws1.cell(row=OriginFinder('Cap',ws1)[0]+1,column=OriginFinder('Cap',ws1)[1]).value

                        # this lot that follow get looped; the r(1-5) is called within the DataSeeker function
                        # defined above

                        ChannelL = DataSeeker(OriginFinder('^Channel Length',ws1),0,0,"{:.2f}",ws1)
                        ChannelW = DataSeeker(OriginFinder('^Channel Length',ws1),0,1,"{:.2f}",ws1)
                        MobilityMed = DataSeeker(OriginFinder('^Channel Length',ws1),0,10,"{:.2f}",ws1)
                        MobSDev = DataSeeker(OriginFinder('^Channel Length',ws1),0,4,"{:.2%}",ws1)
                        Ionoff = DataSeeker(OriginFinder('^IONOFF',ws1),0,8,"{:.2e}",ws1)
                        VTO = DataSeeker(OriginFinder('^VTO$',ws1),0,0,"{:.2f}",ws1)    
                        VTH = DataSeeker(OriginFinder('^VTH',ws1),0,0,"{:.2f}",ws1)
                        Yield = DataSeeker(OriginFinder('^Channel Length',ws1),0,9,"{:.2%}",ws1)

                        Mob_Ind = IndMobs[r-1]
                        VTO_Ind = IndVTOs[r-1]
                        
                        channel_data = (fname,test_date,ChannelL,ChannelW,MobilityMed,
                                        MobSDev,Ionoff,VTO,VTH,Yield,Capacitance,Comments,Mob_Ind,VTO_Ind)

                        substrate_data.append(channel_data)

                        writer.writerow(channel_data)     

                        print(fname,',',test_date,',',ChannelL,',',ChannelW,',',MobilityMed,','
                              ,MobSDev,',',Ionoff,',',VTO,',',VTH,',',Yield,',',Capacitance,',',Mob_Ind, VTO_Ind)   

                        continue

                    print('-'*10,substrate)  ####### DON'T EDIT THIS VERSION !!!!!!!!!!     

        continue 

############## TIMING LOGIC ###############

total_time = ((time.clock() - start_time))
print("--- %s seconds ---" % round(total_time,2))
print("%s seconds per file" % round(total_time/substrate))

###########################################

20160718_NDISFET_R6P4_UCA_L5.xlsm , 2016-07-19 14:48:00 , 4.00 , 30.00 , 1.49 , 20.18% , 1.02e+06 , 19.50 , 2.67 , 86.11% , 6e-09 , [None, None, 1.49329854066, None, 1.74335931311, None, None, 1.59680547514, 1.61663658786, 1.57435377327, 1.70914311066, 1.26970937773, 1.38906230487, 1.4269552288, 1.5002736824, 1.39892649756, 1.45515254381, 1.59141768178, 1.71992519668, 1.5847435853, 1.71453554777, 1.44442153828, 1.38766834964, 1.48335151954, 1.20663694821, 1.31705061395, 1.60968285995, 1.14347548376, 1.47914939093, 1.68234173417, 1.86163650372, None, 1.67613025073, 0.970887223777, 0.688861728548, 1.49467957307] [None, 19.498183, None, 19.498183, None, None, 17.498275, 19.498183, 19.498183, 20.498137, 20.498137, 19.498183, 19.498183, 19.498183, 19.498183, 19.498183, 18.498229000000002, 17.498275, 17.498275, 18.498229000000002, 19.498183, 19.498183, 19.498183, 19.498183, 20.498137, 19.498183, 20.498137, 18.498229000000002, 19.498183, 17.498275, None, 19.498183, 18.498229000000002, 19.4981

C:\Users\ben.coombs\Local_Docs\WinPython\python-3.4.3\lib\site-packages\openpyxl\workbook\names\named_range.py:124: UserWarning: Discarded range with reserved name
  warnings.warn("Discarded range with reserved name")


In [ ]:
coords

In [ ]:
                        VTO1 = []
                        vto1_form = tuple(ws3.iter_rows(coords[0]))
                        for row in vto1_form:
                            for cell in row:
                                VTO1.append(cell.value)
                        VTO2 = []
                        vto2_form = tuple(ws3.iter_rows(coords[1]))
                        for row in vto2_form:
                            for cell in row:
                                VTO2.append(cell.value)
                        VTO3 = []
                        vto3_form = tuple(ws3.iter_rows(coords[2]))
                        for row in vto3_form:
                            for cell in row:
                                VTO3.append(cell.value)
                        VTO4 = []
                        vto4_form = tuple(ws3.iter_rows(coords[3]))
                        for row in vto4_form:
                            for cell in row:
                                VTO4.append(cell.value)

                        IndVTOs = [VTO1, VTO2, VTO3, VTO4]


                        # for loop to return all data for the 4 subsites
                        # with offsets calculated from ORIGINS defined during these calls (using Origin Finder func)                

                        for r in range(1,5):

                            # these first three don't actually loop as are single admin-y data bits

                            test_date = ws1.cell(row=OriginFinder('Date',ws1)[0]+1,column=OriginFinder('Date',ws1)[1]).value
                            Comments = ws1.cell(row=OriginFinder('comment',ws1)[0],column=OriginFinder('comment',ws1)[1]+1).value
                            Capacitance = ws1.cell(row=OriginFinder('Cap',ws1)[0]+1,column=OriginFinder('Cap',ws1)[1]).value

                            # this lot that follow get looped; the r(1-5) is called within the DataSeeker function
                            # defined above

                            ChannelL = DataSeeker(OriginFinder('^Channel Length',ws1),0,0,"{:.2f}",ws1)
                            ChannelW = DataSeeker(OriginFinder('^Channel Length',ws1),0,1,"{:.2f}",ws1)
                            MobilityMed = DataSeeker(OriginFinder('^Channel Length',ws1),0,10,"{:.2f}",ws1)
                            MobSDev = DataSeeker(OriginFinder('^Channel Length',ws1),0,4,"{:.2%}",ws1)
                            Ionoff = DataSeeker(OriginFinder('^IONOFF',ws1),0,8,"{:.2e}",ws1)
                            VTO = DataSeeker(OriginFinder('^VTO$',ws1),0,0,"{:.2f}",ws1)    
                            VTH = DataSeeker(OriginFinder('^VTH',ws1),0,0,"{:.2f}",ws1)
                            Yield = DataSeeker(OriginFinder('^Channel Length',ws1),0,9,"{:.2%}",ws1)

                            Mob_Ind = IndMobs[r-1]
                            VTO_Ind = IndVTOs[r-1]
                            channel_data = (fname,test_date,ChannelL,ChannelW,MobilityMed,
                                            MobSDev,Ionoff,VTO,VTH,Yield,Capacitance,Comments,Mob_Ind,VTO_Ind)

                            substrate_data.append(channel_data)

                            writer.writerow(channel_data)     

                            print(fname,',',test_date,',',ChannelL,',',ChannelW,',',MobilityMed,','
                                  ,MobSDev,',',Ionoff,',',VTO,',',VTH,',',Yield,',',Capacitance,',',Mob_Ind, VTO_Ind)   

                            continue

                        print('-'*10,substrate)  ####### DON'T EDIT THIS VERSION !!!!!!!!!!     

            continue 

############## TIMING LOGIC ###############

total_time = ((time.clock() - start_time))
print("--- %s seconds ---" % round(total_time,2))
print("%s seconds per file" % round(total_time/substrate))

###########################################

In [ ]:
IndVTOs

## Outputs VTO formulae

In [ ]:
import openpyxl
from openpyxl import load_workbook
from openpyxl.cell import get_column_letter, column_index_from_string
import time
import os
import pandas as pd
import csv
import re

start_time = time.clock()

substrate = 0                                               
outputfile = []
channel_data = []
substrate_data = []

date = time.strftime('DIRECTORY TEST DATA OVERVIEW %Y-%m-%d.csv')#-%H%M%S)


def SearchXLWithRE(regex,worksheet):
    for i in range(1, Row_limit):
        for j in range(1, Column_limit):
            query = worksheet.cell(row = i, column = j).value
            if type(query) == str:
                test = regex.search(query)
                if test:                                    
                    return [i,j]                           
    return [100,100]
                

def DataSeeker(ORIGIN,rowOffset,colOffset,X,worksheet):
    result_raw = worksheet.cell(row=ORIGIN[0]+rowOffset+r,
                             column=ORIGIN[1]+colOffset).value
    if not type(result_raw) == str:
        result = X.format(result_raw)
    else:
        result = result_raw
    return result


def OriginFinder(Regex_Term,worksheet):
    Regex_Pattern = re.compile(Regex_Term, re.I)
    Origin_Name = SearchXLWithRE(Regex_Pattern,worksheet)
    return Origin_Name


def filterPick(list,filter):
    return [(m.group(1)) for l in list for m in (filter(l),) if m]
    

for root, dirs, files in os.walk(".", topdown=True):

    for fname in files:

        if fname.endswith('.xlsm') or fname.endswith('.xlsx'):

            wb_formula = load_workbook(os.path.join(root, fname))
            
            names = wb_formula.get_sheet_names()

            if 'validated Summary Data' in names:
                    
                substrate = substrate + 1
                
                wb_value = load_workbook(os.path.join(root, fname), data_only=True)
                
                ws1 = wb_formula.get_sheet_by_name('validated Summary Data')
                ws3 = wb_value.get_sheet_by_name('transferdata')

                Row_limit = ws1.max_row
                Column_limit = ws1.max_column
                Col_limit = get_column_letter(Column_limit)

                for r in range(1,5):

                    VTO = DataSeeker(OriginFinder('^VTO$',ws1),0,0,"{:.2f}",ws1)    
                    channel_data = (VTO)                      
                    substrate_data.append(channel_data)           

                    continue
                
                seeker = re.compile("transferdata!(.*)\),").search
                coords = filterPick(substrate_data,seeker)               

In [ ]:
coords

In [ ]:
substrate_data

In [ ]:
seeker = re.compile("transferdata!(.*)\),").search

In [ ]:
Jeff = tuple(ws3.iter_rows(coords[2]))

In [ ]:
Jeff

In [ ]:
jefftyjeff = []
for row in Jeff:
    for cell in row:
        jefftyjeff.append(cell.value)
    continue

print(jefftyjeff)
print(len(jefftyjeff))